## Import Modules

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Import modules
import os 
import scipy.io
from scipy import stats
from __future__ import print_function
import pandas as pd
from numpy import *

import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from scipy import stats
import pandas as pd

## Load connectome data, organize matrix

In [2]:
test=scipy.io.loadmat('Connectome_group_aparc.a2009s+aseg_AD.mat')
AD_2009=np.array(test['connectome'])
print(AD_2009.shape)

test=scipy.io.loadmat('Connectome_group_aparc.a2009s+aseg_count.mat')
count_2009=np.array(test['connectome'])
print(count_2009.shape)

test=scipy.io.loadmat('Connectome_group_aparc.a2009s+aseg_FA.mat')
FA_2009=np.array(test['connectome'])
print(FA_2009.shape)

test=scipy.io.loadmat('Connectome_group_aparc.a2009s+aseg_length.mat')
length_2009=np.array(test['connectome'])
print(length_2009.shape)

test=scipy.io.loadmat('Connectome_group_aparc.a2009s+aseg_MD.mat')
MD_2009=np.array(test['connectome'])
print(MD_2009.shape)

test=scipy.io.loadmat('Connectome_group_aparc.a2009s+aseg_RD.mat')
RD_2009=np.array(test['connectome'])
print(RD_2009.shape)




test=scipy.io.loadmat('Connectome_group_aparc+aseg_AD.mat')
AD_aseg=np.array(test['connectome'])
print(AD_aseg.shape)

test=scipy.io.loadmat('Connectome_group_aparc+aseg_count.mat')
count_aseg=np.array(test['connectome'])
print(count_aseg.shape)

test=scipy.io.loadmat('Connectome_group_aparc+aseg_FA.mat')
FA_aseg=np.array(test['connectome'])
print(FA_aseg.shape)

test=scipy.io.loadmat('Connectome_group_aparc+aseg_length.mat')
length_aseg=np.array(test['connectome'])
print(length_aseg.shape)

test=scipy.io.loadmat('Connectome_group_aparc+aseg_MD.mat')
MD_aseg=np.array(test['connectome'])
print(MD_aseg.shape)

test=scipy.io.loadmat('Connectome_group_aparc+aseg_RD.mat')
RD_aseg=np.array(test['connectome'])
print(RD_aseg.shape)

zero_AD=np.zeros([164,164,303])
zero_count=np.zeros([164,164,303])
zero_FA=np.zeros([164,164,303])
zero_length=np.zeros([164,164,303])
zero_MD=np.zeros([164,164,303])
zero_RD=np.zeros([164,164,303])

zero_AD[40:124,40:124,:]=AD_aseg
zero_count[40:124,40:124,:]=count_aseg
zero_FA[40:124,40:124,:]=FA_aseg
zero_length[40:124,40:124,:]=length_aseg
zero_MD[40:124,40:124,:]=MD_aseg
zero_RD[40:124,40:124,:]=RD_aseg


X=np.zeros([164,164,303,12])
X[:,:,:,0]=AD_2009
X[:,:,:,1]=count_2009
X[:,:,:,2]=FA_2009
X[:,:,:,3]=length_2009
X[:,:,:,4]=MD_2009
X[:,:,:,5]=RD_2009
X[:,:,:,6]=zero_AD
X[:,:,:,7]=zero_count
X[:,:,:,8]=zero_FA
X[:,:,:,9]=zero_length
X[:,:,:,10]=zero_MD
X[:,:,:,11]=zero_RD


print(X.shape)
X=X.transpose([2,0,1,3])
print(X.shape)

(164, 164, 303)
(164, 164, 303)
(164, 164, 303)
(164, 164, 303)
(164, 164, 303)
(164, 164, 303)
(84, 84, 303)
(84, 84, 303)
(84, 84, 303)
(84, 84, 303)
(84, 84, 303)
(84, 84, 303)
(164, 164, 303, 12)
(303, 164, 164, 12)


## Read in data and labels, organize data for classification

In [3]:
alldata=pd.read_csv('PLA_MOR_136_v2.csv',header=0)
print(alldata.shape)
alldata=np.array(alldata)
datasubjid=alldata[:,0]

list_subjs=pd.read_csv('list_subject_303_connectome.csv',header=0)
#list_subjs=list_subjs.apply(lambda x: x.str.slice(0,6))
print(list_subjs.shape)


filtindex=np.isin(list_subjs,datasubjid)
filtindex=filtindex.ravel()

X=X[filtindex]
print(X.shape)
listofx=list_subjs[filtindex]
#print(listofx)

reverseindex=np.isin(datasubjid,list_subjs)
reverseindex=reverseindex.ravel()
print(reverseindex.shape)

y=alldata[:,1]
y=np.array(y,dtype=np.float64)
y=y[reverseindex]
labelofy=alldata[:,0]
labelofy=labelofy[reverseindex]
print(y.shape)


removeblank=isnan(y)
y_connectome=y[~removeblank]
print(y_connectome.shape)

X=X[~removeblank]
print(X.shape)

newalldata=pd.read_csv('PLA_MOR_136_v2.csv',header=0)
newalldata=np.array(newalldata)
newalldata=newalldata[reverseindex,:]
newalldata=newalldata[~removeblank,:]

Xvar=newalldata[:,(2,4,7,8,25,1047,1048,1049,1050,1051,1052,1053,1054)]
print(Xvar.shape)
Xvar=np.array(Xvar,dtype=np.float32)
wherenan=isnan(Xvar)
Xvar[wherenan]=0

delta_track=newalldata[:,19]-newalldata[:,7]



(136, 1055)
(303, 1)
(134, 164, 164, 12)
(136,)
(134,)
(111,)
(111, 164, 164, 12)
(111, 13)


In [4]:
fill=np.zeros([111,164,164,12])

fill[:,:,:,0]=stats.zscore(X[:,:,:,0])
fill[:,:,:,1]=stats.zscore(X[:,:,:,1])
fill[:,:,:,2]=stats.zscore(X[:,:,:,2])
fill[:,:,:,3]=stats.zscore(X[:,:,:,3])
fill[:,:,:,4]=stats.zscore(X[:,:,:,4])
fill[:,:,:,5]=stats.zscore(X[:,:,:,5])
fill[:,:,:,6]=stats.zscore(X[:,:,:,6])
fill[:,:,:,7]=stats.zscore(X[:,:,:,7])
fill[:,:,:,8]=stats.zscore(X[:,:,:,8])
fill[:,:,:,9]=stats.zscore(X[:,:,:,9])
fill[:,:,:,10]=stats.zscore(X[:,:,:,10])
fill[:,:,:,11]=stats.zscore(X[:,:,:,11])
wherenan=isnan(fill)
fill[wherenan]=0
X_connectome=fill

C:\Users\choyale\AppData\Local\Continuum\miniconda3\envs\tfpls\lib\site-packages\scipy\stats\stats.py:2248: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


## Freesurfer data load in

In [5]:
test=scipy.io.loadmat('fs_group_aparc_all_4_measures_in_matrix.mat')
aparc_free=np.array(test['DATA'])
print(aparc_free.shape)

test=scipy.io.loadmat('fs_group_aparc2009_all_4_measures_in_matrix.mat')
aparc2009_free=np.array(test['DATA'])
print(aparc2009_free.shape)

X=np.concatenate((aparc_free,aparc2009_free),axis=3)
print(X.shape)

X=X.transpose([2,0,1,3])
print(X.shape)

(13, 13, 323, 4)
(13, 13, 323, 4)
(13, 13, 323, 8)
(323, 13, 13, 8)


In [6]:
alldata=pd.read_csv('PLA_MOR_136_v2.csv',header=0)
print(alldata.shape)
alldata=np.array(alldata)
datasubjid=alldata[:,0]

list_subjs=pd.read_csv('list_fs_all_323.csv',header=0)
list_subjs=list_subjs.apply(lambda x: x.str.slice(0,6))
print(list_subjs.shape)



filtindex=np.isin(list_subjs,labelofy)
filtindex=filtindex.ravel()

X=X[filtindex]
print(X.shape)

X=X[~removeblank]
print(X.shape)


(136, 1055)
(323, 1)
(134, 13, 13, 8)
(111, 13, 13, 8)


In [7]:
fill=np.zeros([111,13,13,8])

fill[:,:,:,0]=stats.zscore(X[:,:,:,0])
fill[:,:,:,1]=stats.zscore(X[:,:,:,1])
fill[:,:,:,2]=stats.zscore(X[:,:,:,2])
fill[:,:,:,3]=stats.zscore(X[:,:,:,3])
fill[:,:,:,4]=stats.zscore(X[:,:,:,4])
fill[:,:,:,5]=stats.zscore(X[:,:,:,5])
fill[:,:,:,6]=stats.zscore(X[:,:,:,6])
fill[:,:,:,7]=stats.zscore(X[:,:,:,7])
wherenan=isnan(fill)
fill[wherenan]=0
X_freesurf=fill

C:\Users\choyale\AppData\Local\Continuum\miniconda3\envs\tfpls\lib\site-packages\scipy\stats\stats.py:2248: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


In [8]:
zeropad=np.zeros([111,164,164,8])
zeropad[:,76:89,76:89,:]=X_freesurf
X_freesurf=zeropad
print(X_freesurf.shape)

(111, 164, 164, 8)


## connectome all

In [15]:
Xnew=X_connectome[:,:,:,0:6]
print(Xnew.shape)

(111, 164, 164, 6)


In [20]:
from sklearn.model_selection import RepeatedStratifiedKFold
from CNN_codes.CNN_flexchannel_v import cnn_training
from sklearn.metrics import roc_curve, auc, f1_score
from sklearn import metrics

rskf=RepeatedStratifiedKFold(n_splits=5,n_repeats=1)
store_acc=[]
prediction=[]
truelabel=[]
result_accuracies=[]
result_sensitivity=[]
result_specificity=[]
result_f1s=[]
store_proba=[]
store_predictedv=[]
store_percent=[]

i=0

for train_index,test_index in rskf.split(Xnew,y_connectome):
    i+=1
    print("Number of training points: ",train_index.shape)
    print('Number of testing points: ',test_index.shape)
    X_train,X_test=Xnew[train_index],Xnew[test_index]
    y_train,y_test=y_connectome[train_index],y_connectome[test_index]
    mask_train = np.random.choice(len(X_train),len(X_train), replace=False)
    X_train=X_train[mask_train,:,:,:]
    y_train=y_train[mask_train,]

    
    tf.reset_default_graph()
    record_acc,recordtrainacc,recordvalacc,roc_pred,roc_truelabel,percentage,v_conv_layer,grad_gb,grad_layer,truelabel_val,predlabel_val,v_conv_layer_train,grad_gb_train,grad_layer_train,truelabel_train,predlabel_train=cnn_training(X_train,y_train,X_test,y_test,fc_units=[2000,100,2], 
                 conv_featmap=[5,5,5],l2_norm=0.1,seed=235,
                 learning_rate=1e-2,epoch=200,batch_size=9,imglen=164,channum=6)
    store_acc.append(record_acc)
    prediction.extend(roc_pred)
    truelabel.extend(roc_truelabel)
    #plt.plot(recordtrainacc,label='Training Accuracy')
    #plt.plot(recordvalacc,label="Validation Accuracy")
    #plt.legend()

    scipy.io.savemat('/Users/choyale/Desktop/save_alloutputs/plac_visualize1stlayer_val'+str(i)+'_a2009scon.mat',mdict={'result':v_conv_layer})
    scipy.io.savemat('/Users/choyale/Desktop/save_alloutputs/plac_visualizegradgb_val'+str(i)+'_a2009scon.mat',mdict={'result':grad_gb})
    scipy.io.savemat('/Users/choyale/Desktop/save_alloutputs/plac_visualizegradlayer_val'+str(i)+'_a2009scon.mat',mdict={'result':grad_layer})
    scipy.io.savemat('/Users/choyale/Desktop/save_alloutputs/plac_visualizeinputimages_val'+str(i)+'_a2009scon.mat',mdict={'result':X_test})
    scipy.io.savemat('/Users/choyale/Desktop/save_alloutputs/plac_truelabels_val'+str(i)+'_a2009scon.mat',mdict={'result':truelabel_val})
    scipy.io.savemat('/Users/choyale/Desktop/save_alloutputs/plac_predlabels_val'+str(i)+'_a2009scon.mat',mdict={'result':predlabel_val})

    
    

Number of training points:  (88,)
Number of testing points:  (23,)
Building Network Parameters: 
fc_units=[2000, 100, 2]
l2_norm=0.1
seed=235
learning_rate=0.01
number of batches for training: 9
epoch 1 
Best validation accuracy! iteration:9 accuracy: 56.52173913043478%
epoch 2 
Best validation accuracy! iteration:18 accuracy: 60.869565217391305%
epoch 3 
epoch 4 
epoch 5 
epoch 6 
epoch 7 
epoch 8 
Best validation accuracy! iteration:72 accuracy: 65.21739130434783%
epoch 9 
Best validation accuracy! iteration:81 accuracy: 73.91304347826087%
epoch 10 
epoch 11 
epoch 12 
epoch 13 
epoch 14 
epoch 15 
epoch 16 
epoch 17 
epoch 18 
epoch 19 
epoch 20 
epoch 21 
epoch 22 
epoch 23 
epoch 24 
epoch 25 
epoch 26 
epoch 27 
epoch 28 
epoch 29 
epoch 30 
epoch 31 
epoch 32 
epoch 33 
epoch 34 
epoch 35 
epoch 36 
epoch 37 
epoch 38 
epoch 39 
epoch 40 
epoch 41 
epoch 42 
epoch 43 
epoch 44 
epoch 45 
epoch 46 
epoch 47 
epoch 48 
epoch 49 
epoch 50 
epoch 51 
epoch 52 
epoch 53 
epoch 54 
ep

epoch 15 
epoch 16 
epoch 17 
Best validation accuracy! iteration:170 accuracy: 61.904761904761905%
epoch 18 
epoch 19 
epoch 20 
epoch 21 
epoch 22 
epoch 23 
epoch 24 
epoch 25 
epoch 26 
epoch 27 
epoch 28 
epoch 29 
epoch 30 
epoch 31 
epoch 32 
epoch 33 
epoch 34 
epoch 35 
epoch 36 
epoch 37 
epoch 38 
epoch 39 
epoch 40 
epoch 41 
epoch 42 
epoch 43 
epoch 44 
epoch 45 
epoch 46 
epoch 47 
epoch 48 
epoch 49 
epoch 50 
epoch 51 
epoch 52 
epoch 53 
epoch 54 
Best validation accuracy! iteration:540 accuracy: 66.66666666666666%
epoch 55 
epoch 56 
epoch 57 
epoch 58 
epoch 59 
Best validation accuracy! iteration:590 accuracy: 76.19047619047619%
epoch 60 
epoch 61 
epoch 62 
epoch 63 
epoch 64 
epoch 65 
epoch 66 
epoch 67 
epoch 68 
epoch 69 
Best validation accuracy! iteration:690 accuracy: 85.71428571428571%
epoch 70 
epoch 71 
epoch 72 
epoch 73 
epoch 74 
epoch 75 
epoch 76 
epoch 77 
epoch 78 
epoch 79 
epoch 80 
epoch 81 
epoch 82 
epoch 83 
epoch 84 
epoch 85 
epoch 86 
epo

## morph visualize

In [9]:
fill=np.zeros([111,13,13,8])

fill[:,:,:,0]=stats.zscore(X[:,:,:,0])
fill[:,:,:,1]=stats.zscore(X[:,:,:,1])
fill[:,:,:,2]=stats.zscore(X[:,:,:,2])
fill[:,:,:,3]=stats.zscore(X[:,:,:,3])
fill[:,:,:,4]=stats.zscore(X[:,:,:,4])
fill[:,:,:,5]=stats.zscore(X[:,:,:,5])
fill[:,:,:,6]=stats.zscore(X[:,:,:,6])
fill[:,:,:,7]=stats.zscore(X[:,:,:,7])
wherenan=isnan(fill)
fill[wherenan]=0
X_freesurf=fill

Xnew=X_freesurf[:,:,:,4:8]
print(Xnew.shape)

(111, 13, 13, 4)


C:\Users\choyale\AppData\Local\Continuum\miniconda3\envs\tfpls\lib\site-packages\scipy\stats\stats.py:2248: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


In [10]:
from sklearn.model_selection import RepeatedStratifiedKFold
from CNN_codes.CNN_flexchannel_v import cnn_training
from sklearn.metrics import roc_curve, auc, f1_score
from sklearn import metrics

rskf=RepeatedStratifiedKFold(n_splits=5,n_repeats=1)
store_acc=[]
prediction=[]
truelabel=[]
result_accuracies=[]
result_sensitivity=[]
result_specificity=[]
result_f1s=[]
store_proba=[]
store_predictedv=[]
store_percent=[]

i=0
for train_index,test_index in rskf.split(Xnew,y_connectome):
    i+=1
    print("Number of training points: ",train_index.shape)
    print('Number of testing points: ',test_index.shape)
    X_train,X_test=Xnew[train_index],Xnew[test_index]
    y_train,y_test=y_connectome[train_index],y_connectome[test_index]
    mask_train = np.random.choice(len(X_train),len(X_train), replace=False)
    X_train=X_train[mask_train,:,:,:]
    y_train=y_train[mask_train,]

    
    tf.reset_default_graph()
    record_acc,recordtrainacc,recordvalacc,roc_pred,roc_truelabel,percentage,v_conv_layer,grad_gb,grad_layer,truelabel_val,predlabel_val,v_conv_layer_train,grad_gb_train,grad_layer_train,truelabel_train,predlabel_train=cnn_training(X_train,y_train,X_test,y_test,fc_units=[2000,100,2], 
                 conv_featmap=[5,5,5],l2_norm=0.1,seed=235,
                 learning_rate=1e-2,epoch=200,batch_size=9,imglen=13,channum=4)
    store_acc.append(record_acc)
    prediction.extend(roc_pred)
    truelabel.extend(roc_truelabel)
    #plt.plot(recordtrainacc,label='Training Accuracy')
    #plt.plot(recordvalacc,label="Validation Accuracy")
    #plt.legend()

    scipy.io.savemat('/Users/choyale/Desktop/save_alloutputs/plac_visualize1stlayer_val'+str(i)+'_a2009smorph.mat',mdict={'result':v_conv_layer})
    scipy.io.savemat('/Users/choyale/Desktop/save_alloutputs/plac_visualizegradgb_val'+str(i)+'_a2009smorph.mat',mdict={'result':grad_gb})
    scipy.io.savemat('/Users/choyale/Desktop/save_alloutputs/plac_visualizegradlayer_val'+str(i)+'_a2009smorph.mat',mdict={'result':grad_layer})
    scipy.io.savemat('/Users/choyale/Desktop/save_alloutputs/plac_visualizeinputimages_val'+str(i)+'_a2009smorph.mat',mdict={'result':X_test})
    scipy.io.savemat('/Users/choyale/Desktop/save_alloutputs/plac_truelabels_val'+str(i)+'_a2009smorph.mat',mdict={'result':truelabel_val})
    scipy.io.savemat('/Users/choyale/Desktop/save_alloutputs/plac_predlabels_val'+str(i)+'_a2009smorph.mat',mdict={'result':predlabel_val})
    

C:\Users\choyale\AppData\Local\Continuum\miniconda3\envs\tfpls\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Number of training points:  (88,)
Number of testing points:  (23,)
Building Network Parameters: 
fc_units=[2000, 100, 2]
l2_norm=0.1
seed=235
learning_rate=0.01
number of batches for training: 9
epoch 1 
Best validation accuracy! iteration:9 accuracy: 43.47826086956522%
epoch 2 
epoch 3 
epoch 4 
epoch 5 
epoch 6 
epoch 7 
epoch 8 
Best validation accuracy! iteration:72 accuracy: 47.82608695652174%
epoch 9 
Best validation accuracy! iteration:81 accuracy: 56.52173913043478%
epoch 10 
epoch 11 
epoch 12 
epoch 13 
epoch 14 
epoch 15 
epoch 16 
epoch 17 
epoch 18 
epoch 19 
epoch 20 
epoch 21 
epoch 22 
epoch 23 
epoch 24 
epoch 25 
Best validation accuracy! iteration:225 accuracy: 65.21739130434783%
epoch 26 
epoch 27 
epoch 28 
epoch 29 
epoch 30 
epoch 31 
epoch 32 
epoch 33 
epoch 34 
epoch 35 
epoch 36 
epoch 37 
epoch 38 
epoch 39 
epoch 40 
epoch 41 
epoch 42 
epoch 43 
epoch 44 
ep

number of batches for training: 10
epoch 1 
Best validation accuracy! iteration:10 accuracy: 38.095238095238095%
epoch 2 
Best validation accuracy! iteration:20 accuracy: 42.857142857142854%
epoch 3 
epoch 4 
Best validation accuracy! iteration:40 accuracy: 47.61904761904762%
epoch 5 
Best validation accuracy! iteration:50 accuracy: 52.38095238095238%
epoch 6 
epoch 7 
epoch 8 
epoch 9 
epoch 10 
Best validation accuracy! iteration:100 accuracy: 57.142857142857146%
epoch 11 
Best validation accuracy! iteration:110 accuracy: 66.66666666666666%
epoch 12 
epoch 13 
epoch 14 
epoch 15 
epoch 16 
epoch 17 
epoch 18 
epoch 19 
epoch 20 
epoch 21 
epoch 22 
epoch 23 
epoch 24 
epoch 25 
epoch 26 
epoch 27 
epoch 28 
epoch 29 
epoch 30 
Best validation accuracy! iteration:300 accuracy: 71.42857142857143%
epoch 31 
epoch 32 
epoch 33 
epoch 34 
epoch 35 
epoch 36 
epoch 37 
epoch 38 
epoch 39 
epoch 40 
epoch 41 
epoch 42 
epoch 43 
epoch 44 
epoch 45 
epoch 46 
epoch 47 
epoch 48 
epoch 49 
Be